<a href="https://colab.research.google.com/github/YasminBougammoura/nlp/blob/main/QuantizedModel158A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
%reset -f

In [95]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

In [96]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
eval_iters = 200
n_embd = 32
eps = 1e-3
n_head = 4
n_layer = 4
dropout = 0.2

In [97]:
with open('input.txt','r', encoding = 'utf-8') as f:
    text = f.read()

In [98]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [99]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [100]:
@torch.no_grad()
def estimate_loss():
    out = {}
    qmodel.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = qmodel(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    qmodel.train()
    return out

In [101]:
class BitLinear(nn.Module):

  def __init__(self, in_features, out_features, bits):
    super().__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.bits = bits
    self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
    self.bias = nn.Parameter(torch.Tensor(out_features))
    #nn.init.xavier_uniform_(self.weight) #, a = math.sqrt(5))

    self.reset_parameters()

  def reset_parameters(self):

    stdv = 1.0 / math.sqrt(self.weight.size(1))
    self.weight.data.uniform_(-stdv, stdv)
    self.bias.data.uniform_(-stdv, stdv)

  def quantization(self, input_data):

    abs_mean = torch.mean(torch.abs(self.weight)) + eps
    W = self.weight/abs_mean
    #adj_par = 2**(self.bits-1)/torch.max(torch.abs(input_data))

    if self.training:
      Wq = W + (torch.clip(torch.round(W),-1,1) - W).detach()
    else:
      Wq = torch.clip(torch.round(W),-1,1)

    Xq = abs_mean * input_data
    #Xq = torch.clip(torch.round(input_data*adj_par), -2**(self.bits-1)+eps, 2**(self.bits-1)-eps)

    return Xq,Wq

  def forward(self,x):
    ln = nn.LayerNorm(x.shape[2])
    X_q,W_q = self.quantization(ln(x))

    #gamma = torch.max(torch.abs(x))
    #beta = torch.sum(torch.abs(self.weight))/self.weight.shape[0]*self.weight.shape[1]
    #deq_par =

    y = F.linear(X_q, W_q, self.bias)

    #y = W_q @ X_q.transpose(-2,-1)
    #y = y.transpose(-2,-1)
    #y = y + self.bias.unsqueeze(0)

    return y


In [102]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.key = BitLinear(n_embd, head_size,8)
        self.query = BitLinear(n_embd, head_size,8)
        self.value = BitLinear(n_embd, head_size,8)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * self.head_size **-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [103]:
class MultiHeadAttention(nn.Module):

  def __init__(self,num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = BitLinear(n_embd, n_embd,8)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim = -1) #concat over channel dim
    out = self.proj(out)
    return out

In [104]:
class FeedForward(nn.Module):

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        BitLinear(n_embd,4*n_embd,8), # *4 secondo argomento
        nn.ReLU(),
        BitLinear(4*n_embd, n_embd,8), #proj layer #*4 primo argomento
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.net(x)

In [106]:
class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [114]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        #self.bitlayer = BitLinear(n_embd,vocab_size,8)
        self.linear = nn.Linear(n_embd,vocab_size)


    def forward(self, idx, targets=None):
        B,T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) #(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.linear(x) #self.bitlayer(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [115]:
qmodel = BigramLanguageModel()

In [116]:
optimizer = torch.optim.AdamW(qmodel.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = qmodel(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2602, val loss 4.2597
step 500: train loss 2.4678, val loss 2.4800
step 1000: train loss 2.3595, val loss 2.3727
step 1500: train loss 2.3005, val loss 2.3137
step 2000: train loss 2.2567, val loss 2.2801
step 2500: train loss 2.2282, val loss 2.2520
step 3000: train loss 2.1922, val loss 2.2245
step 3500: train loss 2.1970, val loss 2.2198
step 4000: train loss 2.1700, val loss 2.2108
step 4500: train loss 2.1604, val loss 2.1808


In [117]:
print(decode(qmodel.generate(idx = torch.zeros((1,1),dtype = torch.long), max_new_tokens=500)[0].tolist()))



PULERTOUS:
Head cinaird!, I't whor ful mre, mok
Ce.T MuRCu
That?

FajuR:
And wends: goods leavereas,

Bui hatur met. Prand Sise echom,

Thy
GLorre iy ut hour he'as; woe dige
Tharoun have is for areck,
Therry now Hot dily Her not chat fait ther stul tontran.

KIZAng And bront monace, warte a weld puearvion?

PRD
MOLARUKIA:
No:
Haur reseits
Thou
You a coman!

FEOWBeNG I KIr Rerireg
With borsed mor my magt spaemerm:s of you lee.

CoUning Rarteniath bin is rie vevot Los bat,

On I spird the thou ra
